In [1]:
from os import listdir
import glob
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
filepaths = glob.glob("../data/tidy/acetaminophen/*_events_*.csv")
df = pd.concat(map(pd.read_csv, filepaths))
df["CLINICAL_EVENT_DATETIME"] = pd.to_datetime(df["CLINICAL_EVENT_DATETIME"])
df.index = df["CLINICAL_EVENT_DATETIME"]

/home/brian/miniconda3/envs/med_tracking/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [3]:
df_monthly = df.resample("MS").count()[["EVENT_ID"]]
df_monthly.columns = ["Actual"]

# remove current month
df_monthly = df_monthly[df_monthly.index < datetime.now().strftime('%Y-%m-01')]

In [4]:
from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
from pmdarima import auto_arima

In [5]:
auto_mod = auto_arima(
    df_monthly["Actual"], 
    suppress_warnings=True,
    seasonal=False,
    stepwise=False,
    error_action="ignore"
)

In [6]:
model_arima = ARIMA(df_monthly["Actual"], order=auto_mod.order)
fit_arima = model_arima.fit()

In [7]:
n_pred = 12
fc, se, conf = fit_arima.forecast(n_pred, alpha=0.05)  # 95% conf
idx = pd.date_range(df_monthly.index[-1] + 1, periods = n_pred, freq="MS")
fc_series = pd.Series(fc, index=idx)
lower_series = pd.Series(conf[:, 0], index=idx)
upper_series = pd.Series(conf[:, 1], index=idx)

/home/brian/miniconda3/envs/med_tracking/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df_forecast = pd.DataFrame(fc.round(0), index=idx, columns=["Forecast"])
df_forecast.index.name = "CLINICAL_EVENT_DATETIME"

In [9]:
df_combined = pd.concat([df_monthly, df_forecast]).replace({pd.np.nan: None})

/home/brian/miniconda3/envs/med_tracking/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [27]:
from pptx import Presentation 
from pptx.chart.data import CategoryChartData
from pptx.dml.color import RGBColor
from pptx.enum.chart import XL_CHART_TYPE, XL_LEGEND_POSITION, XL_DATA_LABEL_POSITION, XL_MARKER_STYLE, XL_TICK_MARK
from pptx.enum.dml import MSO_LINE_DASH_STYLE, MSO_THEME_COLOR
from pptx.util import Inches, Pt

In [75]:
prs = Presentation()
# title slide
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]

title.text = "Test Forecast Slides"
subtitle.text = "Updated: Some Date"

In [76]:
# forecast slide
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

chart_data = CategoryChartData()
chart_data.categories = df_combined.index
chart_data.add_series("Actual", df_combined["Actual"])
chart_data.add_series("Forecast", df_combined["Forecast"])

x, y, cx, cy = Inches(1), Inches(1), Inches(8), Inches(6)
chart = slide.shapes.add_chart(XL_CHART_TYPE.LINE, x, y, cx, cy, chart_data).chart


In [77]:
font_nm = "Arial"
ax_bright = 0.35

def format_axis(axis):
    axis.format.line.color.theme_color = MSO_THEME_COLOR.TEXT_1
    axis.format.line.color.brightness = ax_bright
    return axis

def format_axis_title(axis, title_text, has_title = True):
    axis.has_title = True
    axis.axis_title.text_frame.text = title_text
    axis.axis_title.text_frame.paragraphs[0].font.name = font_nm
    axis.axis_title.text_frame.paragraphs[0].font.size = Pt(18)
    axis.axis_title.text_frame.paragraphs[0].font.bold = False
    axis.axis_title.text_frame.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.TEXT_1
    axis.axis_title.text_frame.paragraphs[0].font.color.brightness = ax_bright
    return axis

def format_axis_tick_labels(axis):
    axis.tick_labels.font.name = "Arial"
    axis.tick_labels.font.size = Pt(16)
    axis.tick_labels.font_bold = False
    axis.tick_labels.font.color.theme_color = MSO_THEME_COLOR.TEXT_1
    axis.tick_labels.font.color.brightness = ax_bright
    return axis

def format_marker(chart, i, j, col_mark, col_line):
    chart.series[i].points[j].marker.style = XL_MARKER_STYLE.CIRCLE
    chart.series[i].points[j].marker.format.fill.solid()
    chart.series[i].points[j].marker.format.fill.fore_color.rgb = col_mark
    chart.series[i].points[j].marker.format.line.width = Pt(2.25)
    chart.series[i].points[j].marker.format.line.color.rgb = col_line
    chart.series[i].points[j].data_label.has_text_frame = True
    chart.series[i].points[j].data_label.position = XL_DATA_LABEL_POSITION.BELOW
    chart.series[i].points[j].data_label.text_frame.text = str(round(chart.series[i].values[j], None))
    return chart
    

def format_graph(chart):
    category_axis = format_axis_title(chart.category_axis, "Month")
    category_axis = format_axis_tick_labels(category_axis)
    category_axis = format_axis(category_axis)
    category_axis.tick_labels.number_format = "mmm yy"

    value_axis = format_axis_title(chart.value_axis, "Number of doses")
    value_axis = format_axis_tick_labels(value_axis)
    value_axis = format_axis(value_axis)
    value_axis.has_major_gridlines = False
    value_axis.has_minor_gridlines = False

    chart.has_title = True
    chart.chart_title.text_frame.text = "Acetaminophen IV forecast"
    chart.chart_title.text_frame.paragraphs[0].font.name = font_nm
    chart.chart_title.text_frame.paragraphs[0].font.size = Pt(24)
    chart.chart_title.text_frame.paragraphs[0].font.bold = False
    chart.chart_title.text_frame.paragraphs[0].font.color.theme_color = MSO_THEME_COLOR.TEXT_1
    chart.chart_title.text_frame.paragraphs[0].font.color.brightness = 0.25

    chart.has_legend = False
    # chart.legend.include_in_layout = False
    # chart.legend.position = XL_LEGEND_POSITION.TOP

    # format Actual line
    col_actual = RGBColor.from_string("1f78b4")
    chart.series[0].format.line.width = Pt(3.5)
    chart.series[0].format.line.color.rgb = col_actual
    # chart.series[0].smooth = True

    i = len(df_monthly) - 1
    chart = format_marker(chart, 0, i, RGBColor.from_string("FFFFFF"), col_actual)

    # format Forecast line
    brght_forecast = 0.25

    chart.series[1].format.line.dash_style = MSO_LINE_DASH_STYLE.DASH
    chart.series[1].format.line.width = Pt(2.5)
    chart.series[1].format.line.color.rgb = col_actual
    chart.series[1].format.line.color.brightness = brght_forecast

    j = len(chart.series[1].points) - 1
    chart = format_marker(chart, 1, j, col_actual, col_actual)
    
    return chart

In [78]:
chart = format_graph(chart)

In [80]:
prs.save('../doc/forecast.pptx')